In [1]:
import pandas as pd
import os
import requests
from PIL import Image
import io
import random
import threading
import numpy as np

pd.set_option('max.colwidth', 100)

In [2]:
df = pd.read_csv('../../../data/fashion/data.csv', encoding='utf-8', sep=';')
df.head()

,id,img_path,attributes,brand,category,color,img_url,model_img_urls,name,product_url,sleeve_length,length,neckline,pattern,fit
0,01.899.12.5102_498134,blouses/01.899.12.5102_498134.jpg,"Eingefasster Saum/Kante, Gerader Saum, Lagenlook, Rückenschlitz, Lockerer Schnitt, Unifarben, Ch...",s.Oliver BLACK LABEL,blouses,red,https://cdn.aboutstatic.com/file/e5fc1a5599fb57c91d8d434aa4e92bf0,"https://cdn.aboutstatic.com/file/47daae8d9cde4b7c60afa242eb21f425, https://cdn.aboutstatic.com/f...",Bluse,https://www.aboutyou.de/p/s-oliver-black-label/bluse-3945479,short,normal,NaN,unicolors,loose
1,01.899.12.5104_020034,tops/01.899.12.5104_020034.jpg,"Abgesteppter Saum/Kante, Gerader Saum, Knopfleiste, Saum/Kante mit Borte, Lockerer Schnitt, Unif...",s.Oliver BLACK LABEL,tops,white,https://cdn.aboutstatic.com/file/909575c62c77bea6a5dc50d39325d537,"https://cdn.aboutstatic.com/file/ca6204c741bcc4b3baa4f50d5b8906f0, https://cdn.aboutstatic.com/f...",Bluse,https://www.aboutyou.de/p/s-oliver-black-label/bluse-3945496,NaN,NaN,NaN,unicolors,loose
2,01.899.39.3932_020034,tops/01.899.39.3932_020034.jpg,"Jersey, Rundhals-Ausschnitt, Gerader Saum, Taillierter Schnitt, Unifarben, Strukturierter Griff,...",s.Oliver BLACK LABEL,tops,white,https://cdn.aboutstatic.com/file/991b6f7a4a2ac7c0b562119b09a43d5d,"https://cdn.aboutstatic.com/file/a302b90d651f3f904332739835ec0755, https://cdn.aboutstatic.com/f...",Shirt,https://www.aboutyou.de/p/s-oliver-black-label/shirt-3662143,NaN,NaN,round,unicolors,tight
3,01.899.39.3932_446834,tops/01.899.39.3932_446834.jpg,"Gerader Saum, Taillierter Schnitt, Unifarben, Jersey, Rundhals-Ausschnitt, Strukturierte Oberflä...",s.Oliver BLACK LABEL,tops,pink,https://cdn.aboutstatic.com/file/92e6eba47bf3673b8f29b41531b41612,"https://cdn.aboutstatic.com/file/a2096a4eabb28a7dc625021c4d1d9a64, https://cdn.aboutstatic.com/f...",Langarmshirt mit Allover-Struktur,https://www.aboutyou.de/p/s-oliver-black-label/langarmshirt-mit-allover-struktur-3784925,NaN,NaN,round,unicolors,tight
4,01.899.71.3916_59593034,pants/01.899.71.3916_59593034.jpg,"5-Pocket-Style, Materialmix, Unifarben, Jeans, Label Patch/Label Flag, Steppungen, Ton-in-Ton-Nä...",s.Oliver BLACK LABEL,pants,blue,https://cdn.aboutstatic.com/file/7356873b59eab0403bce7367bfc9a8ed,"https://cdn.aboutstatic.com/file/6e695b4f43c297be92acb7e63c35d692, https://cdn.aboutstatic.com/f...",Jeans,https://www.aboutyou.de/p/s-oliver-black-label/jeans-3571171,NaN,long,NaN,unicolors,tight


In [3]:
df_model_urls = df[['category', 'id', 'model_img_urls']]
df_model_urls['model_img_urls'] = df_model_urls['model_img_urls'].apply(lambda x: str(x).split(', '))
df_model_urls.head()

/Users/sonynka/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,category,id,model_img_urls
0,blouses,01.899.12.5102_498134,"[https://cdn.aboutstatic.com/file/47daae8d9cde4b7c60afa242eb21f425, https://cdn.aboutstatic.com/..."
1,tops,01.899.12.5104_020034,"[https://cdn.aboutstatic.com/file/ca6204c741bcc4b3baa4f50d5b8906f0, https://cdn.aboutstatic.com/..."
2,tops,01.899.39.3932_020034,"[https://cdn.aboutstatic.com/file/a302b90d651f3f904332739835ec0755, https://cdn.aboutstatic.com/..."
3,tops,01.899.39.3932_446834,"[https://cdn.aboutstatic.com/file/a2096a4eabb28a7dc625021c4d1d9a64, https://cdn.aboutstatic.com/..."
4,pants,01.899.71.3916_59593034,"[https://cdn.aboutstatic.com/file/6e695b4f43c297be92acb7e63c35d692, https://cdn.aboutstatic.com/..."


In [4]:
data_folder = '../../../data/fashion_models'
img_width = 256

if not os.path.exists(data_folder):
    os.makedirs(data_folder)

In [5]:
def download_image(img_data, img_filepath, img_width):
    try:
        img = Image.open(io.BytesIO(img_data.content))
        img_ratio = img.size[0] / img.size[1]
        new_size = [img_width, int(img_width/img_ratio)]
        img = img.resize(new_size, Image.ANTIALIAS)
        img.save(img_filepath)
    except:
        print('Problem with downloading image: {}'.format(img_filepath))

In [6]:
def get_response(url):
    
    response = None
    
    try:
        response = requests.get(url, timeout=10)
        return response
    except:
        print("Problem downloading response content for: {} Response Code: {}".format(url, response.status_code))

In [7]:
categories = ['dresses']
last_id = None

In [8]:
def download_imgs(df_imgs, save_path, thread_nr):
    for idx, img_id in enumerate(df_imgs.id):
        if idx % 100 == 0:
            print('{}: {}/{}\n'.format(thread_nr, idx, len(df_imgs.id)))
        model_urls = df_imgs.loc[df_imgs.id == img_id]['model_img_urls'].tolist()[0]
        
        for idx, url in enumerate(model_urls):
            img_path = '{}@{}.jpg'.format(img_id, idx)
            img_path = os.path.join(save_path, img_path)
            
            if not os.path.exists(img_path):
                try:
                    img_data = get_response(url)
                    if img_data.status_code == requests.codes.ok:
                        download_image(img_data, img_path, img_width)
                except Exception as e:
                    print('{}: Problem downloading image {}: {}\n'.format(thread_nr, img_path, e))  

In [9]:
for category in categories:
    print('-' * 50)
    print('Downloading category {}'.format(category))

    cat_folder = os.path.join(data_folder, category)
    if not os.path.exists(cat_folder):
        os.makedirs(cat_folder)
    
    category_urls = df_model_urls[df_model_urls.category == category].sort_values('id')
    
    downloaded_files = [f.split('@')[0] for f in os.listdir(cat_folder)]
    category_urls = category_urls.loc[~category_urls.id.isin(downloaded_files)]
    
    n_splits = 4
    for idx, split in enumerate(np.array_split(category_urls, n_splits)):
        threading.Thread(target=download_imgs, args=(split, cat_folder, idx)).start()

--------------------------------------------------
0: 0/325

1: 0/325

2: 0/325

3: 0/324

0: Problem downloading image ../../../data/fashion_models/dresses/2GU21C06S-Q11@0.jpg: 'NoneType' object has no attribute 'status_code'

0: Problem downloading image ../../../data/fashion_models/dresses/2GU21C06U-Q11@0.jpg: 'NoneType' object has no attribute 'status_code'

0: Problem downloading image ../../../data/fashion_models/dresses/2GU21C073-Q11@0.jpg: 'NoneType' object has no attribute 'status_code'

0: Problem downloading image ../../../data/fashion_models/dresses/2GU21C077-Q11@0.jpg: 'NoneType' object has no attribute 'status_code'

0: Problem downloading image ../../../data/fashion_models/dresses/2GU21C079-Q11@0.jpg: 'NoneType' object has no attribute 'status_code'

0: Problem downloading image ../../../data/fashion_models/dresses/2GU21C07B-Q11@0.jpg: 'NoneType' object has no attribute 'status_code'

0: Problem downloading image ../../../data/fashion_models/dresses/2GU21C07C-Q11@0.jpg: 